# 1. Libraries
---

In [1]:
import torch
import numpy as np
import pandas as pd
import tiktoken
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchinfo import summary
import torch.optim as optim
from torchmetrics import Accuracy
from torchvision import datasets
from torchvision.transforms import ToTensor
import mlflow
import os
from torchmetrics.classification import BinaryAccuracy


from sqlalchemy import create_engine, text

from dotenv import load_dotenv
load_dotenv()

True

# 2. Data
---

In [2]:
#start engine
postgres_database = os.environ["POSTGRES_DATABASE"]
engine = create_engine(postgres_database, echo=True)
table_name = 'messages_spam_detector'

In [3]:
conn = engine.connect()

2025-10-27 00:32:06,839 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-10-27 00:32:06,841 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-27 00:32:06,885 INFO sqlalchemy.engine.Engine select current_schema()
2025-10-27 00:32:06,885 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-27 00:32:06,926 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-10-27 00:32:06,927 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
stmt = text(f"""
    SELECT *
    FROM {table_name}
    WHERE label IS NOT NULL
    ORDER BY id ASC
""")

result = conn.execute(stmt)
data = pd.DataFrame(result)

2025-10-27 00:32:18,547 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-27 00:32:18,547 INFO sqlalchemy.engine.Engine 
    SELECT *
    FROM messages_spam_detector
    WHERE label IS NOT NULL
    ORDER BY id ASC

2025-10-27 00:32:18,548 INFO sqlalchemy.engine.Engine [generated in 0.00142s] {}


In [5]:
data.head()

,id,content,label,label_predict,created_at
0,0,"Go until jurong point, crazy.. Available only ...",ham,None,1761506577863
1,1,Ok lar... Joking wif u oni...,ham,None,1761506577863
2,2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,None,1761506577863
3,3,U dun say so early hor... U c already then say...,ham,None,1761506577863
4,4,"Nah I don't think he goes to usf, he lives aro...",ham,None,1761506577863


# 3. Preprocess
---

In [ ]:
# RUsing 1 and 0 for binary classification
data['label_bin'] = data['label'].apply(lambda x: 1  if x =='Spam' else 0)

#tokenization
tokenizer = tiktoken.get_encoding("cl100k_base")
def encode_texts(texts):
    return [tokenizer.encode(text) for text in texts]

data['content_tokenised'] = encode_texts(data['content'])

#sequence length distribution
seq_lens = [len(seq) for seq in data['content_tokenised']]
print(f"avg seq len {np.mean(seq_lens)}")
print(f"max seq len {np.max(seq_lens)}")
#standardise length distribution

def pad_sequences(sequences, max_length=25):
    return [seq[:max_length] + [0] * (max_length - len(seq)) for seq in sequences]

data['content_tokenised'] = pad_sequences(data['content_tokenised'])

avg seq len 23.238924356742118
max seq len 260


# 4. Training
---

In [ ]:
class SpamDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = torch.tensor(texts, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.texts)
        
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

#Creating a dataset instance
df_dataset = SpamDataset(data['content_tokenised'], data['label_bin'])

# Split dataset into training (80%) and validation (20%)
train_size = int(0.8 * len(df_dataset))
val_size = len(df_dataset) - train_size
train_dataset, val_dataset = random_split(df_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


text, label = next(iter(train_loader))

In [71]:
vocab_size = tokenizer.n_vocab

#model defition
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(embed_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)
        pooled = self.pooling(embedded.permute(0, 2, 1)).squeeze(2)
        return torch.sigmoid(self.fc(pooled))

model = TextClassifier(vocab_size=vocab_size,
                      embed_dim=32, 
                      num_class=1)


#describe model
print(model)
summary(model, input_data=text)  # (batch_size, input_features)

TextClassifier(
  (embedding): Embedding(100277, 32, padding_idx=0)
  (pooling): AdaptiveAvgPool1d(output_size=1)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
TextClassifier                           [32, 1]                   --
├─Embedding: 1-1                         [32, 25, 32]              3,208,864
├─AdaptiveAvgPool1d: 1-2                 [32, 32, 1]               --
├─Linear: 1-3                            [32, 1]                   33
Total params: 3,208,897
Trainable params: 3,208,897
Non-trainable params: 0
Total mult-adds (M): 102.68
Input size (MB): 0.01
Forward/backward pass size (MB): 0.21
Params size (MB): 12.84
Estimated Total Size (MB): 13.05

In [72]:
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
experiment_name = 'SMS_Spam_Detector'

mlflow.set_experiment(experiment_name)

2025/10/26 15:52:46 INFO mlflow.tracking.fluent: Experiment with name 'SMS_Spam_Detector' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://fp-private-bucket/mlflow/9', creation_time=1761490366984, experiment_id='9', last_update_time=1761490366984, lifecycle_stage='active', name='SMS_Spam_Detector', tags={}>

In [73]:
# Get cpu or gpu for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [74]:
def train(dataloader, model, loss_fn, metrics_fn, optimizer, epoch):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X).squeeze(1)
        loss = loss_fn(pred, y)  # y float
        accuracy = metrics_fn(pred, y.int())  # y int pour metric

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            step = batch + epoch * len(dataloader)
            mlflow.log_metric("loss", loss.item(), step=step)
            mlflow.log_metric("accuracy", accuracy.item(), step=step)
            print(f"Epoch {epoch+1}, batch {batch}, loss={loss.item():.4f}, acc={accuracy.item():.4f}")

In [ ]:
def evaluate(dataloader, model, loss_fn, metrics_fn, epoch):
    model.eval()
    eval_loss, eval_accuracy = 0.0, 0.0
    num_batches = len(dataloader)

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X).squeeze(1)  
            eval_loss += loss_fn(pred, y).item()
            eval_accuracy += metrics_fn(pred, y).item()

    eval_loss /= num_batches
    eval_accuracy /= num_batches
    mlflow.log_metric("eval_loss", eval_loss, step=epoch)
    mlflow.log_metric("eval_accuracy", eval_accuracy, step=epoch)

    print(f"Eval: loss={eval_loss:.4f}, acc={eval_accuracy:.4f}")

In [76]:
epochs = 20
loss_fn = nn.BCELoss()
metric_fn = BinaryAccuracy().to(device)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [77]:
with mlflow.start_run() as run:
  params = {
      "epochs": epochs,
      "learning_rate": 1e-3,
      "batch_size": 32,
      "loss_function": loss_fn.__class__.__name__,
      "metric_function": metric_fn.__class__.__name__,
      "optimizer": optimizer.__class__.__name__,
  }
  # Log training parameters.
  mlflow.log_params(params)

  # Log model summary.
  with open("model_summary.txt", "w") as f:
      f.write(str(summary(model)))
  mlflow.log_artifact("model_summary.txt")

  for t in range(epochs):
      print(f"Epoch {t + 1} \
-------------------------------")
      train(train_loader, model, loss_fn, metric_fn, optimizer, epoch=t)
      evaluate(val_loader, model, loss_fn, metric_fn, epoch=0)

  # Save the trained model to MLflow.
  model_info = mlflow.pytorch.log_model(model, name="SMS_Spam_Detector")

Exception during reset or similar
Traceback (most recent call last):
  File "/home/lyxor/miniconda3/envs/DP/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 985, in _finalize_fairy
    fairy._reset(
  File "/home/lyxor/miniconda3/envs/DP/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 1433, in _reset
    pool._dialect.do_rollback(self)
  File "/home/lyxor/miniconda3/envs/DP/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 711, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: SSL connection has been closed unexpectedly

Exception during reset or similar
Traceback (most recent call last):
  File "/home/lyxor/miniconda3/envs/DP/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 985, in _finalize_fairy
    fairy._reset(
  File "/home/lyxor/miniconda3/envs/DP/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 1433, in _reset
    pool._dialect.do_rollback(self)
  File "/home/lyxor/miniconda3/envs/DP/lib/python3

Epoch 1 -------------------------------
Epoch 1, batch 0, loss=0.7023, acc=0.4062
Epoch 1, batch 100, loss=0.6496, acc=0.8438
Eval: loss=0.6313, acc=0.8491
Epoch 2 -------------------------------
Epoch 2, batch 0, loss=0.6270, acc=0.8438
Epoch 2, batch 100, loss=0.6030, acc=0.8438
Eval: loss=0.5450, acc=0.8843
Epoch 3 -------------------------------
Epoch 3, batch 0, loss=0.5590, acc=0.8438
Epoch 3, batch 100, loss=0.5363, acc=0.8125
Eval: loss=0.4624, acc=0.8890
Epoch 4 -------------------------------
Epoch 4, batch 0, loss=0.4291, acc=0.8750
Epoch 4, batch 100, loss=0.3901, acc=0.9375
Eval: loss=0.3913, acc=0.8900
Epoch 5 -------------------------------
Epoch 5, batch 0, loss=0.3462, acc=0.9062
Epoch 5, batch 100, loss=0.3156, acc=0.9375
Eval: loss=0.3353, acc=0.9013
Epoch 6 -------------------------------
Epoch 6, batch 0, loss=0.3977, acc=0.8438
Epoch 6, batch 100, loss=0.3030, acc=0.9062
Eval: loss=0.2912, acc=0.9061
Epoch 7 -------------------------------
Epoch 7, batch 0, loss=0

2025/10/26 15:56:57 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/26 15:57:00 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/26 15:57:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run treasured-loon-39 at: https://lyx51-mlflow.hf.space/#/experiments/9/runs/601969b1f09541c69051039a56eb8ef2
🧪 View experiment at: https://lyx51-mlflow.hf.space/#/experiments/9


# 5. Test model
---

In [6]:
input = data['content'][2]
input

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's   "

In [7]:
def preprocess(input):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    token = tokenizer.encode(input)
    seq = [token[:25] + [0] * (25 - len(token))]
    return seq


In [122]:
# after training
import numpy
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
outputs = loaded_model.predict(np.array(preprocess(input)))
outputs

array([[0.9696844]], dtype=float32)

In [ ]:
# with register model

mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
model_name = "SMS_Spam_Detector_NN"
model_version = "latest"

model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")

predict = model.predict(np.array(preprocess(input)))
predict

array([[0.9696844]], dtype=float32)

In [ ]:
if predict[0][0] > 0.96:
    print('Spam')
else:
    print("Not Spam")

spam


In [ ]:
def request_prediction(model, input):
    prediction = model.predict(np.array(preprocess(input)))
    return 'Spam' if prediction[0][0] > 0.5 else 'Not Spam'

In [19]:
request_prediction(model, input)

'spam'

In [ ]:
#def request_prediction(model, input):
#    prediction = model.predict(np.array(preprocess(input)))
 #   return 'spam' if prediction[0][0] > 0.5 else 'ham'